note that julia code can be run for free at:
https://juliabox.com/

In [1]:
using JuMP
using Cbc

In [2]:
m_rows = 6
n_cols = 7

# both should be integers

assert(m_rows < n_cols)


In [3]:
# this was also posted online, and saved in "diet_problem_recut_somewhat.ipynb"

# idea, for a short wide matrix -- i.e. full row rank but underdetermined sytem of equations -- 
# perhaps we want to select a solution vector x, in the form Ax = b, but we want (a non-unique) x 
# that has the minimum L1 norm
# note that using primal simplex means our resulting vector should be sparse
# note that we have closed form / analytic solutions for the case of underdetermined system of equations, 
# and we want to minimize the L2 norm of the solution vector x
# however for the L1 norm case, it is just a linear programming approach

mymodel = Model(solver= CbcSolver())


A = randn(m_rows, n_cols) 
b = randn(m_rows)
# iid standard normal r.v.'s populate these matrices
# note that (in standard form) there is zero probability of A not having full row rank


@variable(mymodel,    x[i = 1:n_cols] ) 
@variable(mymodel,    y[i = 1:n_cols] >= 0) 
@constraint(mymodel, -y .<= x)
@constraint(mymodel,  y .>= x)
# in effect y is the absolute value version of x

@constraint(mymodel, *(A,x) .== b)
# always need to maintain this equality

@objective(mymodel, Min, sum(y))
# minimize the L1 norm

# Solve the optimization problem
solve(mymodel)
println("Variable values: \n", getvalue(x))


Variable values: 
[-1.98388,-0.309694,-0.309281,-1.22742,0.0,-0.825472,0.509495]


# below is a comparison of the underdetermined minimization case with L1 vs L2 norms

In [4]:
U, sigma, V = svd(A)

(
[0.445762 0.487639 … -0.292342 -0.0669869; 0.506457 -0.301959 … 0.589905 -0.350976; … ; 0.199931 -0.547471 … -0.739684 -0.196366; 0.110381 0.576564 … -0.128926 -0.0120897],

[3.35666,3.16647,1.54796,1.15415,1.03224,0.481671],
[0.0646093 0.325034 … 0.684249 -0.326994; -0.158646 0.324937 … 0.0389622 -0.249159; … ; 0.783792 -0.434592 … 0.257422 0.0780215; 0.00572256 -0.438057 … -0.398205 -0.613171])

In [5]:
P = V * transpose(V)

7×7 Array{Float64,2}:
  0.954636     -0.071975    -0.0516647    …  -0.000107481  -0.100424   
 -0.071975      0.885804    -0.0819714       -0.00017053   -0.159333   
 -0.0516647    -0.0819714    0.94116         -0.000122409  -0.114371   
  0.0136366     0.0216359    0.0155305        3.23091e-5    0.0301876  
  0.1587        0.251794     0.180742         0.000376008   0.351318   
 -0.000107481  -0.00017053  -0.000122409  …   1.0          -0.000237933
 -0.100424     -0.159333    -0.114371        -0.000237933   0.77769    

In [6]:
diagm(eigvals(P))

7×7 Array{Float64,2}:
 -2.71187e-17  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          1.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0  1.0  0.0  0.0  0.0  0.0
  0.0          0.0  0.0  1.0  0.0  0.0  0.0
  0.0          0.0  0.0  0.0  1.0  0.0  0.0
  0.0          0.0  0.0  0.0  0.0  1.0  0.0
  0.0          0.0  0.0  0.0  0.0  0.0  1.0

In [7]:
loose_x = diagm(1./ sigma)transpose(U)*b




6-element Array{Float64,1}:
 -0.206303 
 -0.0639741
 -0.143272 
 -0.564088 
 -2.21688  
  0.548479 

In [8]:
A * loose_x

LoadError: DimensionMismatch("second dimension of A, 7, does not match length of x, 6")

In [9]:
x_l2_norm_min = V*diagm(1./ sigma)transpose(U)*b


7-element Array{Float64,1}:
 -1.47869 
 -0.674534
 -0.363262
 -1.25303 
 -0.298017
 -0.82527 
  0.698077

In [10]:
# x_l2_norm_min_alt = transpose(A)*inv(A*transpose(A))*b

In [11]:
P = V * transpose(V)

7×7 Array{Float64,2}:
  0.954636     -0.071975    -0.0516647    …  -0.000107481  -0.100424   
 -0.071975      0.885804    -0.0819714       -0.00017053   -0.159333   
 -0.0516647    -0.0819714    0.94116         -0.000122409  -0.114371   
  0.0136366     0.0216359    0.0155305        3.23091e-5    0.0301876  
  0.1587        0.251794     0.180742         0.000376008   0.351318   
 -0.000107481  -0.00017053  -0.000122409  …   1.0          -0.000237933
 -0.100424     -0.159333    -0.114371        -0.000237933   0.77769    

In [12]:
# x_l2_norm_min


In [13]:
norm(x_l2_norm_min)

2.3665957835042786

In [14]:
norm(getvalue(x) - x_l2_norm_min)

0.39996341257139534

In [15]:
norm(P*getvalue(x) - x_l2_norm_min)

2.410027994385663e-14

In [16]:
Q,R = qr(transpose(A))


(
[-0.169027 0.216392 … 0.501613 0.393796; -0.380116 -0.383837 … -0.0742746 -0.673681; … ; -0.31731 0.853826 … 0.0238207 -0.384782; 0.244248 0.242186 … -0.740516 0.184695],

[-2.4002 -0.483234 … 0.616476 -1.15182; 0.0 2.07049 … 1.3493 -0.756634; … ; 0.0 0.0 … -1.21866 0.525979; 0.0 0.0 … 0.0 1.39879])

In [17]:
Q

7×6 Array{Float64,2}:
 -0.169027    0.216392   -0.22129    -0.650831   0.501613    0.393796
 -0.380116   -0.383837   -0.168739   -0.325805  -0.0742746  -0.673681
  0.62357     0.0839992  -0.599209    0.206447   0.277219   -0.258344
  0.518929    0.0855573   0.62575    -0.408122   0.189506   -0.353914
  0.0920458   0.0615992  -0.412578   -0.405261  -0.28491    -0.130048
 -0.31731     0.853826    0.0169814   0.146225   0.0238207  -0.384782
  0.244248    0.242186   -0.0378726  -0.274715  -0.740516    0.184695

In [18]:
R

6×6 Array{Float64,2}:
 -2.4002  -0.483234   0.81221   -0.773722   0.616476  -1.15182 
  0.0      2.07049   -0.452478   1.48815    1.3493    -0.756634
  0.0      0.0       -1.59131    0.88885   -0.07272    0.459871
  0.0      0.0        0.0        0.700369   0.676543  -0.234243
  0.0      0.0        0.0        0.0       -1.21866    0.525979
  0.0      0.0        0.0        0.0        0.0        1.39879 

In [19]:
Q*(transpose(R) \ b)

7-element Array{Float64,1}:
 -1.47869 
 -0.674534
 -0.363262
 -1.25303 
 -0.298017
 -0.82527 
  0.698077

In [20]:
Q*inv(transpose(R))*b

7-element Array{Float64,1}:
 -1.47869 
 -0.674534
 -0.363262
 -1.25303 
 -0.298017
 -0.82527 
  0.698077

In [21]:
x_l2_norm_min

7-element Array{Float64,1}:
 -1.47869 
 -0.674534
 -0.363262
 -1.25303 
 -0.298017
 -0.82527 
  0.698077

# Below is the overdetermined, L1 minimization case

In [22]:
m_rows = 10
n_cols = 5

# both should be integers


5

In [23]:
# the snow plowing problem, part B
# this was also posted online, and saved in "diet_problem_recut_somewhat.ipynb"

# idea, for a short wide matrix -- i.e. full row rank but underdetermined sytem of equations -- 
# perhaps we want to select a solution vector x, in the form Ax = b, but we want (a non-unique) x 
# that has the minimum L1 norm
# note that using primal simplex means our resulting vector should be sparse
# note that we have closed form / analytic solutions for the case of underdetermined system of equations, 
# and we want to minimize the L2 norm of the solution vector x
# however for the L1 norm case, it is just a linear programming approach

using JuMP

using Cbc
mymodel = Model(solver= CbcSolver())


A = randn(m_rows, n_cols) 
b = randn(m_rows)
# iid standard normal r.v.'s populate these matrices
# note that (in standard form) there is zero probability of A not having full row rank


@variable(mymodel,    x[i = 1:n_cols] ) 
@variable(mymodel,    y[i = 1:m_rows] >= 0) 
@constraint(mymodel, -y .<= (*(A,x) - b))
@constraint(mymodel,  y .>= (*(A,x) - b))
# in effect y is the absolute value version of right hand side

# @constraint(mymodel, *(A,x) - b)
# always need to maintain this equality

@objective(mymodel, Min, sum(y))
# minimize the L1 norm

# Solve the optimization problem
solve(mymodel)
println("Variable values: \n", getvalue(x))


Variable values: 
[-0.362761,0.378531,0.502986,-0.0362082,-0.0974588]


In [24]:
getvalue(x)

5-element Array{Float64,1}:
 -0.636761 
  0.378531 
  0.502986 
 -0.0261082
 -0.0974588

In [25]:
getvalue(y)

10-element Array{Float64,1}:
 0.655348
 0.0     
 2.00452 
 1.28478 
 0.0     
 0.0     
 1.81891 
 0.0     
 0.0     
 0.307033

In [26]:
b

10-element Array{Float64,1}:
 -1.1102  
 -0.331686
  0.775374
  1.30504 
 -0.128578
 -0.488185
 -1.40913 
 -1.3082  
 -0.115728
 -0.109336

In [27]:
*(A,getvalue(x)) - b

10-element Array{Float64,1}:
  0.655348   
  5.55112e-17
 -2.00452    
 -1.28478    
 -1.11022e-16
  1.11022e-16
  1.81891    
  2.22045e-16
  2.63678e-16
  0.307033   

In [28]:
sum(abs(*(A,getvalue(x)) - b))

6.070590520548784

# L2 min for underdetermined not full row rank 

below are some misc things for quadratic program with short fat A, but NOT full row rank... I think I can use svd to minimize this, though some care is needed.  First I'll use a quadratic program. 

In [43]:
m = 15
n = 25
assert(m< n)
A = rand(m,n)

# A[1,:] = 1*A[2,:] + 4*A[3,:]
A
# hence not full row rank as row 1 is linearly dependent 
# hence not enough linearly independent columns to form a basis in m dim space (because row rank = col rank)
b =rand(m)
size(A)

(15,25)

In [44]:
A

15×25 Array{Float64,2}:
 0.495632   0.209378  0.883235   …  0.719306   0.102873   0.0423098  
 0.870124   0.358779  0.0733163     0.328484   0.211271   0.427075   
 0.233821   0.163064  0.849157      0.513849   0.28942    0.89629    
 0.0393918  0.204541  0.0427985     0.0203672  0.0125367  0.000271776
 0.5617     0.133024  0.879119      0.688967   0.392684   0.532432   
 0.979312   0.784037  0.826692   …  0.822639   0.313866   0.777543   
 0.215199   0.765507  0.301955      0.520607   0.891951   0.147879   
 0.180398   0.480473  0.0977619     0.864383   0.796425   0.629041   
 0.396905   0.502633  0.690414      0.496385   0.171607   0.511884   
 0.809465   0.535923  0.326417      0.0621264  0.867563   0.548158   
 0.131879   0.321873  0.782405   …  0.283863   0.536261   0.815521   
 0.684168   0.889901  0.877514      0.809757   0.223899   0.0658682  
 0.981292   0.88661   0.44906       0.234338   0.813761   0.200482   
 0.925511   0.743849  0.1558        0.489562   0.964374   0.096910

In [45]:
F = svdfact(A, thin= false)
# svd fact allows you to pass F[:U], F[:S], F[:V] and F[:Vt], which is rather convenient
F_thin = svdfact(A, thin= true)

# http://web.mit.edu/julia_v0.6.2/julia/share/doc/julia/html/en/stdlib/linalg.html



Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}([-0.267009 -0.229746 … -0.0993284 0.0787569; -0.209784 0.401255 … 0.269971 -0.329404; … ; -0.251937 0.38893 … 0.105049 0.252458; -0.258331 -0.0415683 … -0.116362 0.324435],[10.1812,2.2577,2.13138,1.96943,1.83703,1.66019,1.53515,1.35913,1.14945,1.06736,0.926299,0.749373,0.707779,0.524059,0.334708],[-0.215191 -0.196966 … -0.178918 -0.16025; 0.0344002 -0.0375692 … 0.137 0.0145237; … ; 0.367655 -0.0316973 … -0.141169 -0.198811; -0.190725 0.266555 … 0.124263 -0.0222828])

In [57]:
c = inv(diagm(F[:S]))* transpose(F[:U])* b

15-element Array{Float64,1}:
 -0.159532  
  0.191238  
  0.240676  
  0.109905  
  0.00884959
 -0.164614  
  0.222767  
  0.0117791 
  0.162426  
  0.255315  
  0.432351  
 -0.47617   
 -0.429123  
  0.149368  
 -0.296344  

In [55]:
err_tol = 0.001
V_mod = F_thin[:V] *diagm(F_thin[:S].>err_tol)
P = V_mod * transpose(V_mod)
P

25×25 Array{Float64,2}:
  0.665581     0.124314    -0.0175731    …  -0.0430159    -0.0729055 
  0.124314     0.517406     0.100675         0.202069     -0.176486  
 -0.0175731    0.100675     0.729679        -0.000428068   0.0499664 
  0.105318     0.0599262   -0.0113483        0.0478096     0.0761988 
  0.0790931   -0.130685     0.0303454        0.158058     -0.0667847 
  0.0822263    0.124146    -0.169811     …  -0.105391      0.0476215 
  0.173981    -0.00940229   0.0369311       -0.0487983     0.0433466 
  0.10758     -0.106538     0.0490891        0.0372905     0.0685303 
 -0.0111416    0.0906556   -0.147484         0.0562985     0.00364628
 -0.144266     0.0202958    0.0244906       -0.100658     -0.15103   
 -0.0390231    0.00973952  -0.0170731    …   0.126269     -0.0472705 
 -0.0622012   -0.039056    -0.138601        -0.126184      0.0658346 
  0.0976407    0.167723     0.0748265       -0.136773      0.0583324 
 -0.200545     0.0126508    0.176592        -0.0763       -0.09687

In [64]:
# this is the quadratic setup.  
# encapsulating this in a function would likely lead to better performance, 
# though the 'open' setup here allows the user to more easily explore results

using JuMP
using Ipopt
mymodel = Model(solver=IpoptSolver(print_level=0))


@variable(mymodel, x[i = 1:n])

difference_vec = A*x - b

if rank(A) < minimum(size(A))
    println("ill specified optimization")
    # so if we don't have full row rank, then just minimize total squared errors cost of both the size of x
    # and also the difference vec... 
    # this is worth thinking on some more... 
    myobjective = @objective(mymodel, Min, (transpose(difference_vec)* difference_vec)[1] + (transpose(x) * x)[1] )
else
    # if we do have full row rank, make sure that the solution is exact, and minimize the cost of x
    println("fully specified optimization for full row rank case")
    @constraint(mymodel, (transpose(difference_vec)* difference_vec)[1] == 0)
    myobjective = @objective(mymodel, Min,  + (transpose(x) * x)[1] )
    
end

# Solve mymodel
solve(mymodel)

# print("note: ignore floating point nits -- e.g. 0.9999999976 should be thought of as 1.0")
print("\n--------------------------------------------------\n")

println("Variable x:Values: ", getvalue(x))

println("Objective value: ", getobjectivevalue(mymodel), "\n")

fully specified optimization for full row rank case

--------------------------------------------------
Variable x:Values: [0.237065,0.15406,0.0705198,-0.301197,-0.369363,-0.088939,0.059965,-0.12275,0.503728,-0.123466,0.0649557,-0.278245,0.036335,0.0763259,0.0776803,0.186946,0.0376133,0.0131876,0.299168,-0.0851564,0.384999,0.110396,0.0137056,-0.0261452,-0.107083]
Objective value: 1.0084749123370131



In [65]:
c

15-element Array{Float64,1}:
 -0.159532  
  0.191238  
  0.240676  
  0.109905  
  0.00884959
 -0.164614  
  0.222767  
  0.0117791 
  0.162426  
  0.255315  
  0.432351  
 -0.47617   
 -0.429123  
  0.149368  
 -0.296344  

In [70]:
F_thin[:Vt]*getvalue(x)

15-element Array{Float64,1}:
 -0.159532  
  0.191238  
  0.240676  
  0.109905  
  0.00884959
 -0.164614  
  0.222767  
  0.0117791 
  0.162426  
  0.255314  
  0.432351  
 -0.476169  
 -0.429123  
  0.149368  
 -0.296343  

In [72]:
F[:Vt]*getvalue(x)

25-element Array{Float64,1}:
 -0.159532   
  0.191238   
  0.240676   
  0.109905   
  0.00884959 
 -0.164614   
  0.222767   
  0.0117791  
  0.162426   
  0.255314   
  0.432351   
 -0.476169   
 -0.429123   
  0.149368   
 -0.296343   
 -7.67776e-10
  1.30162e-8 
 -6.10437e-9 
  1.22672e-8 
 -8.94684e-9 
 -7.28173e-10
  4.40785e-9 
 -6.35031e-9 
 -8.72045e-9 
  5.58992e-9 

In [61]:
F[:V]*getvalue(x)

25-element Array{Float64,1}:
 -0.281543   
  0.11066    
  0.0658214  
 -0.000865727
 -0.400783   
 -0.0421855  
  0.057246   
 -0.120863   
 -0.140768   
 -0.158874   
  0.0259013  
  0.234533   
  0.282602   
 -0.228078   
 -0.313895   
  0.0516091  
 -0.267173   
  0.0647632  
 -0.0291125  
 -0.133085   
  0.496979   
  0.0254383  
 -0.130732   
 -0.197182   
 -0.0280272  

In [34]:
# difference_vec_after_projection = P*getvalue(x) - getvalue(x)
difference_vec_after_projection = (P - eye(size(P)[1]))*getvalue(x)
norm(difference_vec_after_projection)

3.0881886493281345e-8

In [35]:
F[:Vt]*getvalue(x)

25-element Array{Float64,1}:
 -0.234459   
 -0.0676163  
 -0.0312089  
 -0.168893   
  0.0321733  
 -0.150168   
 -0.00629015 
  0.358185   
  0.307708   
  0.221686   
 -0.0984953  
  0.74154    
  0.475333   
  0.103012   
  1.07623    
 -3.37631e-10
 -7.12851e-10
  1.73857e-8 
  2.20976e-9 
  1.60355e-9 
 -2.17283e-8 
  7.53564e-9 
  2.06828e-9 
  1.42318e-9 
 -1.03992e-8 

In [36]:
A*getvalue(x) - b

15-element Array{Float64,1}:
  1.59325e-7
 -2.53344e-7
 -3.94036e-7
  2.20123e-8
  6.22103e-8
  4.39108e-7
 -4.61283e-7
 -9.26487e-8
 -2.82391e-7
  1.4668e-7 
 -2.35755e-7
 -3.00236e-7
  2.5771e-8 
  5.68394e-7
  5.46145e-7

In [37]:
print(rank(A))
sum(abs(F[:Vt]*getvalue(x)) .> err_tol)

15

15

In [38]:
# here is a simple L1 norm minimizing attempt to solve the underdetermined system of equations -- i.e. get a qualifying x


mymodel = Model(solver= CbcSolver())


@variable(mymodel,    x_other[i = 1:n] ) 
@variable(mymodel,    y[i = 1:m] >= 0) 
@constraint(mymodel, -y .<= *(A,x_other)- b)
@constraint(mymodel,  y .>= *(A,x_other)- b)
# in effect y is the absolute value version of x

@objective(mymodel, Min, sum(y))
# minimize the L1 norm

# Solve the optimization problem
solve(mymodel)
println("Variable values: \n", getvalue(x_other))


Variable values: 
[1.80611e12,3.13868e12,1.29458e13,2.07006e13,3.01237e12,1.17654e12,8.81925e12,-3.31566e13,2.12496e13,4.32131e12,-4.98955e12,1.67068e12,-4.4373e12,1.69751e12,4.19848e12,1.63666e13,1.92822e12,-1.10044e13,-1.27716e13,1.43185e13,-4.01737e13,-5.17508e12,-8.03078e12,7.26387e12,5.32892e12]


In [39]:
norm(A*getvalue(x_other) - b)

0.02564339342117616

In [40]:
getvalue(x_other)

25-element Array{Float64,1}:
  1.80611e12
  3.13868e12
  1.29458e13
  2.07006e13
  3.01237e12
  1.17654e12
  8.81925e12
 -3.31566e13
  2.12496e13
  4.32131e12
 -4.98955e12
  1.67068e12
 -4.4373e12 
  1.69751e12
  4.19848e12
  1.63666e13
  1.92822e12
 -1.10044e13
 -1.27716e13
  1.43185e13
 -4.01737e13
 -5.17508e12
 -8.03078e12
  7.26387e12
  5.32892e12

In [41]:
P*getvalue(x_other)

25-element Array{Float64,1}:
  0.256592 
 -0.209137 
 -0.279785 
  0.177124 
 -0.0190125
 -0.266357 
  0.712402 
 -0.0906067
  0.0768661
 -0.244507 
 -0.0335693
 -0.163574 
 -0.236633 
  0.0379562
  0.0567627
  0.198669 
 -0.548616 
  0.463196 
  0.489746 
  0.320313 
 -0.0193481
  0.405212 
  0.450195 
 -0.15435  
 -0.0131836

In [42]:
getvalue(x)

25-element Array{Float64,1}:
  0.274713 
 -0.203582 
 -0.283221 
  0.171458 
 -0.0223745
 -0.276543 
  0.719975 
 -0.0971316
  0.0766921
 -0.265628 
 -0.0500203
 -0.170985 
 -0.225619 
  0.0393252
  0.0583854
  0.216452 
 -0.568365 
  0.473077 
  0.489513 
  0.343962 
 -0.0105203
  0.395922 
  0.460224 
 -0.149768 
 -0.0337528